In [1]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

In [2]:
# Load the CSV file
df = pd.read_csv('12th Gen.csv')

# Ensure the text column is of type string
df['text'] = df['text'].astype(str)

In [3]:
# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Function to preprocess text
def preprocess_text(text):
    return tokenizer(text, padding=True, truncation=True, return_tensors='pt')

# Tokenize the text
tokens = [preprocess_text(text) for text in df['text']]


In [4]:
# Load the pre-trained model for sentiment analysis
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')


In [5]:
# Function to predict sentiment
def predict_sentiment(tokenized_input):
    with torch.no_grad():
        outputs = model(**tokenized_input)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        return predicted_class

# Function to map predicted class to sentiment label
def sentiment_label(predicted_class):
    labels = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
    return labels[predicted_class]

In [6]:
df['sentiment_numeric'] = [predict_sentiment(tokens[i]) for i in range(len(tokens))]

# Map the numeric sentiment to descriptive sentiment
df['sentiment_description'] = df['sentiment_numeric'].apply(sentiment_label)


In [8]:
print(df[['text', 'sentiment_numeric', 'sentiment_description']].head(10))

                                                text  sentiment_numeric  \
0  15k is very low price for an i3. 15.1k should ...                  1   
1    Works great out the box and super quiet cooler.                  2   
2  This is a great processor that can handle just...                  2   
3  I bought this processor to be able to© put it ...                  2   
4  12100 is a great CPU because Intel did bust a ...                  2   
5     Great budget processor for not too heavy work.                  2   
6  It's great for when you're on a budget, has go...                  2   
7  I have an i7-10700K. In Single-Core he beats 1...                  2   
8               working well until© today march 2024                  2   
9  Came in early.. box damaged but luckily cpu wa...                  2   

  sentiment_description  
0               Neutral  
1              Positive  
2              Positive  
3              Positive  
4              Positive  
5              Pos

In [9]:
# Save the dataframe to a new CSV file
df.to_csv('robertabased_with_sentiment_12th.csv', index=False)